In [ ]:
#DRAFT 1 DOESNT WORK
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense
class SimplexAttention(Layer):
    def __init__(self, units):
        super(SimplexAttention, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W_q = Dense(self.units)
        self.W_k = Dense(self.units)
        self.W_v = Dense(self.units)

    def call(self, query, key, value, mask):
        # Linear layers
        query = self.W_q(query)
        key = self.W_k(key)
        value = self.W_v(value)

        # Scale dot-product attention
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)

        # Apply mask
        if mask is not None:
            logits += (mask * -1e9)

        # Softmax
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)

        return output


class DuplexAttention(SimplexAttention):
    def call(self, query, key, value, mask):
        # Apply Simplex Attention
        simplex_output = super().call(query, key, value, mask)

        # Apply Duplex Attention (considering future states)
        future_mask = None  # Not applying the mask for future tokens
        duplex_output = super().call(query, key, value, future_mask)

        # Combine simplex and duplex outputs
        output = (simplex_output + duplex_output) / 2

        return output

class BipartiteTransformer(Layer):
    def __init__(self, units):
        super(BipartiteTransformer, self).__init__()
        self.simplex_attention = SimplexAttention(units)
        self.duplex_attention = DuplexAttention(units)
        self.linear = Dense(units)

    def call(self, inputs, mask=None):
        simplex_output = self.simplex_attention(inputs, inputs, inputs, mask)
        duplex_output = self.duplex_attention(inputs, inputs, inputs, mask)

        # Combine the outputs
        combined_output = tf.concat([simplex_output, duplex_output], axis=-1)

        # Pass through a linear layer
        output = self.linear(combined_output)

        return output


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

#DRAFT 2
def positional_encoding(position, d_model):
    angle_rads = tf.range(position, dtype=tf.float32)[:, tf.newaxis] / tf.math.pow(10000, (tf.range(d_model, dtype=tf.float32)[tf.newaxis, :] // 2) * 2 / d_model) #wtf i just vomited pure garbage
    angle_rads[:, 0::2] = tf.math.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = tf.math.cos(angle_rads[:, 1::2])
    return angle_rads[tf.newaxis, ...]

class SimplexAttention(layers.Layer):
    def __init__(self, d_model):
        super(SimplexAttention, self).__init__()
        self.d_model = d_model
        self.W_q = layers.Dense(d_model)
        self.W_k = layers.Dense(d_model)
        self.W_v = layers.Dense(d_model)

    def call(self, query, key, value):
        query = self.W_q(query)
        key = self.W_k(key)
        value = self.W_v(value)

        scores = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        attention_weights = tf.nn.softmax(scores, axis=-1)

        output = tf.matmul(attention_weights, value)
        return output

class DuplexAttention(layers.Layer):
    def __init__(self, d_model):
        super(DuplexAttention, self).__init__()
        self.d_model = d_model
        self.W_q = layers.Dense(d_model)
        self.W_k = layers.Dense(d_model)
        self.W_v = layers.Dense(d_model)

    def call(self, query, key, value):
        query = self.W_q(query)
        key = self.W_k(key)
        value = self.W_v(value)

        # Adding positional encoding
        pos_encoding = positional_encoding(key.shape[1], self.d_model)
        key += pos_encoding
        value += pos_encoding

        scores = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        attention_weights = tf.nn.softmax(scores, axis=-1)

        future_output = tf.matmul(attention_weights, value)

        return future_output
"""
class BipartiteTransformer(layers.Layer):
    def __init__(self, d_model):
        super(BipartiteTransformer, self).__init__()
        self.simplex_attention = SimplexAttention(d_model)
        self.duplex_attention = DuplexAttention(d_model)
        self.feed_forward = layers.Dense(d_model)

    def call(self, query, key, value):
        simplex_output = self.simplex_attention(query, key, value)
        duplex_output = self.duplex_attention(query, key, value)

        # Merge the Simplex and Duplex outputs.
        merged_output = tf.concat([simplex_output, duplex_output], axis=-1)

        output = self.feed_forward(merged_output)
        return output

"""
input_data = tf.random.normal((32, 50, 128))  # (batch_size, sequence_length, d_model)
bipartite_transformer = BipartiteTransformer(d_model=128)
output = bipartite_transformer(input_data, input_data, input_data)

print("Bipartite Transformer Output Shape:", output.shape)


In [ ]:
from tensorflow.keras import layers, models
import tensorflow as tf

def build_RDNN(input_shape=(50, 10)):
    input_layer = layers.Input(shape=input_shape)

    x = layers.LSTM(128, return_sequences=True)(input_layer)
    x = layers.Dropout(0.2)(x)
    x = layers.LSTM(64, return_sequences=True)(x)
    x = layers.Dropout(0.2)(x)

    output_layer = layers.Dense(input_shape[1])(x)  # Output shape is the same as input shape on the last dimension (10)

    model = models.Model(inputs=input_layer, outputs=output_layer)

    return model


In [ ]:
from tensorflow.keras import layers, models

def build_mha_wgan_generator():
    input_layer = layers.Input(shape=(5, 256, 256, 1))  # 5 images of shape 256x256x1

    x = layers.TimeDistributed(layers.Flatten())(input_layer)

    x = layers.LSTM(512, return_sequences=True)(x)

    # Multi-Head Attention Layer
    mha_layer = layers.MultiHeadAttention(num_heads=4, key_dim=512)
    x = mha_layer(x, x)

    x = layers.LSTM(1024, return_sequences=True)(x)

    x = layers.Reshape((5, 16, 16, 16))(x)

    x = layers.Conv3DTranspose(128, (3, 3, 3), strides=(1, 2, 2), padding='same', activation='relu')(x)

    x = layers.Conv3DTranspose(64, (3, 3, 3), strides=(1, 2, 2), padding='same', activation='relu')(x)

    output_images = layers.Conv3DTranspose(1, (3, 3, 3), strides=(1, 2, 2), padding='same', activation='tanh')(x)

    return models.Model(inputs=input_layer, outputs=output_images)

generator = build_mha_wgan_generator()
generator.summary()

def build_mha_wgan_discriminator():
    input_layer = layers.Input(shape=(5, 256, 256, 1))  # 5 images of shape 256x256x1

    x = layers.Conv3D(64, (3, 3, 3), strides=(1, 2, 2), padding='same', activation='relu')(input_layer)

    x = layers.Flatten()(x)

    # Multi-Head Attention Layer
    mha_layer = layers.MultiHeadAttention(num_heads=4, key_dim=512)
    x = mha_layer(x, x)

    x = layers.Dense(1024, activation='relu')(x)

    # Using linear activation for Wasserstein loss
    output_layer = layers.Dense(1)(x)

    return models.Model(inputs=input_layer, outputs=output_layer)

discriminator = build_mha_wgan_discriminator()
discriminator.summary()


In [ ]:
# idk this is a janky model so im trying everything
import tensorflow as tf
from tensorflow.keras import layers, models


class BipartiteTransformer(layers.Layer):
    def __init__(self, units):
        super(BipartiteTransformer, self).__init__()
        self.simplex_attention = SimplexAttention()
        self.duplex_attention = DuplexAttention()

    def call(self, x):
        simplex_out = self.simplex_attention(x, x, x)  # Query, Key, Value
        duplex_out = self.duplex_attention(x, x, x)  # Query, Key, Value
        return layers.Add()([simplex_out, duplex_out])
'''
def build_cGATN_generator(mha_wgan_gen, rdnn):
    # Process image output from MHA WGAN
    image_processed = layers.Conv2D(64, (3,3), padding='same', activation='relu')(mha_wgan_gen.output)
    image_processed = layers.MaxPooling2D((2, 2))(image_processed)
    image_processed = layers.Flatten()(image_processed)

    # Process numerical data output from RDNN
    num_processed = layers.Dense(50, activation='relu')(rdnn.output)

    # Merge processed image and numerical data
    merged = layers.Concatenate()([image_processed, num_processed])

    # Processing fused data with LSTM
    lstm_out = layers.LSTM(50, return_sequences=True)(layers.Reshape((-1, 1))(merged))  # Adding sequence dimension
    lstm_out = layers.LSTM(50)(lstm_out)

    # Integrating Bipartite Transformer
    transformer = BipartiteTransformer(units=50)
    transformer_out = transformer(tf.expand_dims(lstm_out, axis=1))  # Adding time dimension


    x = layers.Dense(256, activation="relu")(transformer_out)
    x = layers.BatchNormalization()(x)
    x = layers.Reshape((16, 16, 1))(x)
    x = layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    output_image = layers.Conv2DTranspose(1, (3,3), strides=(2,2), padding='same', activation='tanh')(x)

    return models.Model(inputs=[mha_wgan_gen.input, rdnn.input], outputs=output_image)
'''

# Assuming the output shape from RDNN is (50,10)
# Assuming the output shape from MHA WGAN is (5, 256, 256, 1)

def build_cGATN_generator(mha_wgan_gen, rdnn):
    # Process sequence of 5 images output from MHA WGAN
    image_processed = layers.TimeDistributed(layers.Conv2D(64, (3,3), padding='same', activation='relu'))(mha_wgan_gen.output)
    image_processed = layers.TimeDistributed(layers.MaxPooling2D((2, 2)))(image_processed)
    image_processed = layers.TimeDistributed(layers.Flatten())(image_processed)

    # Process numerical data output from RDNN
    num_processed = layers.Dense(50, activation='relu')(rdnn.output)

    # Expand dimensions to merge with images
    num_processed = layers.RepeatVector(5)(num_processed)

    # Merge processed image and numerical data
    merged = layers.Concatenate(axis=-1)([image_processed, num_processed])

    # Processing fused data with LSTM
    lstm_out = layers.LSTM(50, return_sequences=True)(merged)
    lstm_out = layers.LSTM(50, return_sequences=True)(lstm_out)

    # Integrating Bipartite Transformer
    transformer = BipartiteTransformer(units=100) #50 * 2
    transformer_out = transformer(lstm_out)

    # Completing the generator architecture to produce an image as output
    x = layers.TimeDistributed(layers.Dense(256, activation="relu"))(transformer_out)
    x = layers.TimeDistributed(layers.BatchNormalization())(x)
    x = layers.TimeDistributed(layers.Reshape((16, 16, 1)))(x)
    x = layers.TimeDistributed(layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu'))(x)
    x = layers.TimeDistributed(layers.BatchNormalization())(x)
    output_image = layers.TimeDistributed(layers.Conv2DTranspose(1, (3,3), strides=(2,2), padding='same', activation='tanh'))(x)

    return models.Model(inputs=[mha_wgan_gen.input, rdnn.input], outputs=output_image)


mha_wgan_gen = build_mha_wgan_generator()
rdnn = build_RDNN()

# Now build the cGATN generator
cGATN_gen = build_cGATN_generator(mha_wgan_gen, rdnn)
cGATN_gen.summary()


def build_cGATN_discriminator():
    radar_input = layers.Input(shape=(64, 64, 1))
    num_weather_data_input = layers.Input(shape=(50,))

    # Fuse numerical weather data with radar imagery
    x = layers.Flatten()(radar_input)
    merged = layers.Concatenate()([x, num_weather_data_input])

    # Building the discriminator
    x = layers.Dense(256, activation="relu")(merged)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    output_layer = layers.Dense(1, activation="sigmoid")(x)

    return models.Model(inputs=[radar_input, num_weather_data_input], outputs=output_layer)



In [ ]:
# STILL FLESHING THIS PART OUT
import tensorflow as tf

# Hyperparameters
batch_size = 32
epochs = 100
learning_rate = 0.00005

# Build models
mha_wgan_gen, mha_wgan_disc = build_MHA_WGAN()
rdnn = build_RDNN()
cGATN_gen, cGATN_disc = build_cGATN_generator(mha_wgan_gen, rdnn), build_cGATN_discriminator()

# Compile models
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# Custom Wasserstein loss function
def wasserstein_loss(y_true, y_pred):
    return -tf.reduce_mean(y_true * y_pred)

mha_wgan_gen.compile(optimizer=optimizer, loss=wasserstein_loss)
mha_wgan_disc.compile(optimizer=optimizer, loss=wasserstein_loss)
rdnn.compile(optimizer=optimizer, loss='mse')  # Mean squared error, or another suitable loss
cGATN_gen.compile(optimizer=optimizer, loss=wasserstein_loss)
cGATN_disc.compile(optimizer=optimizer, loss=wasserstein_loss)


# X_train_images, X_train_numerical_data and y_train_images

for epoch in range(epochs):
    for (X_images, X_numerical, y_images) in zip(X_train_images, X_train_numerical_data, y_train_images):

        # Labels for real and fake data
        real_labels = -tf.ones((batch_size, 1))
        fake_labels = tf.ones((batch_size, 1))

        # Train MHA WGAN
        with tf.GradientTape() as tape:
            generated_images = mha_wgan_gen(X_images)
            real_output = mha_wgan_disc(y_images)
            fake_output = mha_wgan_disc(generated_images)
            disc_loss = wasserstein_loss(real_labels, real_output) + wasserstein_loss(fake_labels, fake_output)

        grads = tape.gradient(disc_loss, mha_wgan_disc.trainable_variables)
        optimizer.apply_gradients(zip(grads, mha_wgan_disc.trainable_variables))

        with tf.GradientTape() as tape:
            generated_images = mha_wgan_gen(X_images)
            fake_output = mha_wgan_disc(generated_images)
            gen_loss = wasserstein_loss(real_labels, fake_output)

        grads = tape.gradient(gen_loss, mha_wgan_gen.trainable_variables)
        optimizer.apply_gradients(zip(grads, mha_wgan_gen.trainable_variables))

        # Train RDNN
        rdnn_loss = rdnn.train_on_batch(X_numerical, y_numerical)  # Assuming y_numerical is the target

        # Train cGATN
        refined_images = cGATN_gen([generated_images, rdnn_output])
        cGATN_disc_loss = cGATN_disc.train_on_batch([refined_images, rdnn_output], real_labels)
        cGATN_gen_loss = cGATN_gen.train_on_batch([refined_images, rdnn_output], fake_labels)

        print(f"Epoch: {epoch}, MHA WGAN Discriminator Loss: {disc_loss}, MHA WGAN Generator Loss: {gen_loss}")
        print(f"Epoch: {epoch}, cGATN Discriminator Loss: {cGATN_disc_loss}, cGATN Generator Loss: {cGATN_gen_loss}, RDNN Loss: {rdnn_loss}")

